In [ ]:
import vtk
import gcsfs
import os
import tempfile

# Cloud Storage details
bucket_name = "marketplace-2xim6sjc" 
file_path_in_gcs = "TotalSegmentator v2/99/artifactFiles/s0000/segmentations/femur_left.nii.gz"
project_id = 'pbeat03002010-prod-d8b6-pp-def'

# Access Cloud Storage
fs = gcsfs.GCSFileSystem(project=project_id)

# Create a temporary directory
temp_dir = tempfile.mkdtemp()
local_file_path = os.path.join(temp_dir, 'temp_segmentation.nii.gz')

# Download the file to the temporary location
fs.get(f'gs://{bucket_name}/{file_path_in_gcs}', local_file_path)

# Now use the local file path with VTK
dataImporter = vtk.vtkNIFTIImageReader()
dataImporter.SetFileName(local_file_path) 
dataImporter.Update()

# Extract the femur segmentation
imageData = dataImporter.GetOutput()

# Create a contour filter to extract the surface of the segmented femur
contourFilter = vtk.vtkContourFilter()
contourFilter.SetInputData(imageData)
contourFilter.SetValue(0, 1)  # Assuming the femur segmentation is labeled as 1
contourFilter.Update()

# Create a mapper and actor
mapper = vtk.vtkPolyDataMapper()
mapper.SetInputConnection(contourFilter.GetOutputPort())
actor = vtk.vtkActor()
actor.SetMapper(mapper)

# Create a renderer, render window, and interactor
renderer = vtk.vtkRenderer()
renderWindow = vtk.vtkRenderWindow()
renderWindow.AddRenderer(renderer)
renderWindowInteractor = vtk.vtkRenderWindowInteractor()
renderWindowInteractor.SetRenderWindow(renderWindow)

# Add the actor to the scene
renderer.AddActor(actor)
renderer.SetBackground(0.2, 0.3, 0.4)  # Set background color

# Initialize the interactor and start the rendering loop
renderWindowInteractor.Initialize()
renderWindow.Render()
renderWindowInteractor.Start()

# Clean up the temporary file (optional but recommended)
os.remove(local_file_path)
os.rmdir(temp_dir)